In [1]:
include("prepare_data_Sioux.jl")

tapFlowVecToLinkCostDict (generic function with 1 method)

In [2]:
include("../../Julia_files/inverseVI_multi_class.jl")

train (generic function with 1 method)

In [3]:
#Read in the demand file

file = open("../../data_original/SiouxFalls_trips.txt")

demands = Dict()
demands_car = Dict()
demands_truck = Dict()

s = 0
for line in eachline(file)
    if contains(line, "Origin")
        s = parse(Int, split(line)[2])
    else
        pairs = split(line, ";")
        for pair in pairs
            if !contains(pair, "\n")
                pair_vals = split(pair, ":")
                t, demand = parse(Int, pair_vals[1]), parse(Float64, pair_vals[2])
                demands[(s,t)] = demand
                demands_car[(s,t)] = demand * .8
                demands_truck[(s,t)] = demand * .2
            end
        end
    end
end

close(file)

In [4]:
#read in the arc file

arcs = arcData("./data_Sioux/SiouxFalls_net.txt");

In [5]:
tap_flows = readstring("./data_Sioux/SiouxFalls_flows.json");
tap_flows = JSON.parse(tap_flows);

In [6]:
#read in the car flows and truck flows

for key in keys(tap_flows["car"])
    key_ = (parse(Int, split(split(key, ",")[1], "(")[2]),parse(Int, split(split(key, ",")[2], ")")[1]))
    arcs[key_].flow_car = tap_flows["car"][key]
    arcs[key_].flow_truck = tap_flows["truck"][key]
end


#calculate the total flow on each link

for (s,t) in keys(arcs)
    arcs[(s,t)].flow = 1.0 * arcs[(s,t)].flow_car + 2.0 * arcs[(s,t)].flow_truck
end

In [7]:
coeffs_dict = Dict()

deg_list = [3, 4, 5, 6]
c_list = [0.5, 1.0, 1.5]
lam_list = [0.01, 0.1, 1.0, 10.0, 100.0]

for deg in deg_list
    for c in c_list
        for lam in lam_list
            coeffs_dict[(deg, c, lam)], _ = train(lam, deg, c, demands_car, demands_truck, arcs)
        end
    end
end

In [8]:
outfile = open("./results_Sioux/coeffs_dict_Sioux.json", "w")

JSON.print(outfile, coeffs_dict)

close(outfile)